In [1]:
from pathlib import Path

import spiceypy as spice
from yarl import URL
from planetarypy.spice import (
    datasets,
    download_generic_kernels,
    get_metakernel_and_files,
    list_kernels_for_day,
    load_generic_kernels,
)

In [2]:
datasets.head()

,Archive Readme,Archive Link,PDS3 or PDS4,Data Size (GB),Start Time,Stop Time,Subset Link
Mission Name,,,,,,,
BepiColombo,https://naif.jpl.nasa.gov/pub/naif/pds/pds4/bc...,https://naif.jpl.nasa.gov/pub/naif/pds/pds4/bc...,4,4.5,2018-10-20,2025-05-31,https://naif.jpl.nasa.gov/cgi-bin/subsetds.pl?...
CLPS,https://naif.jpl.nasa.gov/pub/naif/pds/pds4/cl...,https://naif.jpl.nasa.gov/pub/naif/pds/pds4/cl...,4,0.2,2024-01-08,2024-02-22,https://naif.jpl.nasa.gov/cgi-bin/subsetds.pl?...
Cassini Orbiter,https://naif.jpl.nasa.gov/pub/naif/pds/data/co...,https://naif.jpl.nasa.gov/pub/naif/pds/data/co...,3,73.5,1997-10-15,2017-09-15,https://naif.jpl.nasa.gov/cgi-bin/subsetds.pl?...
Clementine,https://naif.jpl.nasa.gov/pub/naif/pds/data/cl...,https://naif.jpl.nasa.gov/pub/naif/pds/data/cl...,3,0.8,1994-01-26,1994-05-07,https://naif.jpl.nasa.gov/cgi-bin/subsetds.pl?...
DART,https://naif.jpl.nasa.gov/pub/naif/pds/pds4/da...,https://naif.jpl.nasa.gov/pub/naif/pds/pds4/da...,4,10.1,2021-11-09,2050-01-01,https://naif.jpl.nasa.gov/cgi-bin/subsetds.pl?...


In [3]:
def last_part(path, n):
    """Show only the last n parts from a pathlib.Path object."""
    # this is useful to remove machine-specific leading paths
    # and keep the notebook clean even when coded on different machines
    return Path(*path.parts[-n:]) if n > 0 else path

# NAIF Archived SPICE Kernels
Below table shows all officially archived SPICE kernels from NAIF. 
These kernels are less often updated and are not guaranteed to be the most recent versions.

However, they are still useful for many applications and can be used as a stable reference.

Some ongoing missions like MRO and LRO regularly update their archived kernels, but not very frequently.

The `datasets` object is just a parsed table from the NAIF website [here](https://naif.jpl.nasa.gov/naif/data_archived.html).

In [10]:
datasets.loc["BepiColombo"].values

array(['https://naif.jpl.nasa.gov/pub/naif/pds/pds4/bc/bc_spice/document/spiceds_v005.html',
       'https://naif.jpl.nasa.gov/pub/naif/pds/pds4/bc/bc_spice', '4',
       '4.5', '2018-10-20', '2025-05-31',
       'https://naif.jpl.nasa.gov/cgi-bin/subsetds.pl?dataset=bc/bc_spice'],
      dtype=object)

In [11]:
list_kernels_for_day?

Signature: list_kernels_for_day(mission: str, start: str, stop: str = '') -> list
Docstring:
List all kernels for a given time range of a mission.

Parameters
----------
mission : str
    Mission shorthand in datasets dataframe.
start : str
    Start time in either ISO or yyyy-jjj format.
stop : str, optional
    Stop time in either ISO or yyyy-jjj format. Defaults to None.
File:      ~/Dropbox/src/planetarypy/src/planetarypy/spice/archived_kernels.py
Type:      function

In [12]:
list_kernels_for_day("messenger", "2011-01-01")

KeyError: 'messenger'

## Most useful function `get_metakernel_and_files()`:

For a given mission and time-range, it downloads and stores all relevant archived kernels in a local archive, edits the provided meta-kernel from NAIF to point to the correct local paths, and returns the path to the local meta-kernel, ready to be loaded with spiceypy.furnsh().

In [20]:
mk_path = get_metakernel_and_files(
    mission="cassini", start="2015-01-01", stop="2015-01-02"
)
last_part(mk_path, 3)

Kernels downloaded:   0%|          | 0/31 [00:00<?, ?it/s]

PosixPath('spice_kernels/cassini/cas_2015_v09_150101_150102.tm')

As you can see, the meta-kernel can be loaded immediately with spiceypy:

In [25]:
spice.furnsh(str(mk_path))  # spiceypy requires a string path

### Generic Kernels
The below functions download and load a few generic kernels that enable some basic SPICE calculations with the major planetary bodies.

TODO: Script a test that shows for which bodies some basic calculations can be done.

In [26]:
load_generic_kernels()  # uses below download_generic_kernels if any kernel from generic list is missing.

In [27]:
download_generic_kernels()

naif0012.tls already downloaded. Use `overwrite=True` to download again.
pck00010.tpc already downloaded. Use `overwrite=True` to download again.
de-403-masses.tpc already downloaded. Use `overwrite=True` to download again.
de430.bsp already downloaded. Use `overwrite=True` to download again.
mar097.bsp already downloaded. Use `overwrite=True` to download again.


In [ ]:
# Trim URL columns to last two path components for cleaner display
from urllib.parse import urlparse

def _last_n_segments(u: str | None, n: int = 2) -> str | None:
    if not u or not isinstance(u, str):
        return None
    p = urlparse(u)
    parts = [seg for seg in p.path.split('/') if seg]
    if not parts:
        return None
    return '/".join(parts[-n:])

# Overwrite any "... URL" columns with shortened paths
for col in df_split.columns:
    if col.endswith(" URL"):
        df_split[col] = df_split[col].apply(lambda u: _last_n_segments(u, 2))

df_split.head()